# Process single cell morphology features for CellProfiler readouts - PyBaSiC and CellProfiler Cellpose plugin Method

## Import Libraries

In [1]:
import pathlib
import pandas as pd

from pycytominer import normalize, feature_select
from pycytominer.cyto_utils import cells, output

## Set up paths to CellProfiler directory and outputs

In [2]:
# Set file and directory constants
cp_dir = "../../../CellProfiler_pipelines"
output_dir = "../../data/Plate2/CellProfiler"

## Set up paths to sqlite files and outputs

In [3]:
# Set name and path of .sqlite file and path to metadata
sql_file_pbcellpose = "NF1_data_pybasic_cellpose_plate2.sqlite"
single_cell_file_pbcellpose = f"sqlite:///{cp_dir}/Analysis_Output/Plate2_Output/{sql_file_pbcellpose}"
platemap_file = f"{cp_dir}/Metadata/platemap_NF1_CP_Plate2.csv"

# Set path with name for outputted data
sc_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_pybasic_cellpose.csv.gz")
sc_norm_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_norm_pybasic_cellpose.csv.gz")
sc_norm_fs_output_file_pbcellpose = pathlib.Path(f"{output_dir}/nf1_sc_norm_fs_pybasic_cellpose.csv.gz")

## Set up names for linking columns between tables in the database file

In [4]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_Cytoplasm": {
        "Per_Cells": "Cytoplasm_Parent_Cells",
        "Per_Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Per_Cells": {"Per_Cytoplasm": "Cells_Number_Object_Number"},
    "Per_Nuclei": {"Per_Cytoplasm": "Nuclei_Number_Object_Number"},
}

## Load in platemap

In [5]:
# Load platemap file
platemap_df = pd.read_csv(platemap_file)
platemap_df

,WellRow,WellCol,well_position,gene_name,genotype
0,A,1,A1,NF1,WT
1,A,6,A6,NF1,WT
2,A,7,A7,NF1,Null
3,A,12,A12,NF1,Null
4,B,1,B1,NF1,WT
5,B,6,B6,NF1,WT
6,B,7,B7,NF1,Null
7,B,12,B12,NF1,Null
8,C,1,C1,NF1,WT
9,C,6,C6,NF1,WT


## Set up `SingleCells` class from Pycytominer

In [6]:
# Instantiate SingleCells class
sc_pbcellpose = cells.SingleCells(
    sql_file=single_cell_file_pbcellpose,
    compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/4.process-nf1-features/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_cells, per_cytoplasm, per_nuclei
  warnings.warn(warn_str)


## Merge single cells 

In [7]:
# Merge single cells across compartments
anno_kwargs = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df_pbcellpose = sc_pbcellpose.merge_single_cells(
    platemap=platemap_file,
    **anno_kwargs,
)

# Save level 2 data as a csv
output(sc_df_pbcellpose, sc_output_file_pbcellpose)

print(sc_df_pbcellpose.shape)
sc_df_pbcellpose.head()

(1741, 1207)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,A,1,NF1,WT,7,001,A1,1,5,1,...,330.386081,320.255565,57.928927,60.619538,71.79197,58.919416,57.774974,56.582428,57.449434,57.448597
1,A,1,NF1,WT,7,001,A1,2,6,2,...,452.816916,447.257847,87.652345,77.648563,83.016288,92.922156,19.82313,20.017045,19.916222,19.828628
2,A,1,NF1,WT,7,001,A1,3,7,3,...,128.117331,135.330565,98.270691,100.825745,113.866931,106.715129,71.807042,68.650079,70.141523,70.357582
3,A,1,NF1,WT,7,001,A1,4,8,4,...,287.824126,276.300597,43.624512,46.146436,50.825451,43.530095,30.964635,30.43804,31.592918,31.602776
4,A,1,NF1,WT,7,001,A1,5,9,5,...,431.973964,434.235182,58.831491,56.362605,64.704343,56.210369,53.207779,49.403182,50.535135,49.849143


## Normalize data

In [8]:
# Normalize single cell data and write to file
normalize_sc_pbcellpose = normalize(
    sc_df_pbcellpose,
    method="standardize"
)

output(normalize_sc_pbcellpose, sc_norm_output_file_pbcellpose)

print(normalize_sc_pbcellpose.shape)
normalize_sc_pbcellpose.head()

(1741, 1207)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,A,1,NF1,WT,7,001,A1,1,5,1,...,-0.615375,-0.619527,-0.703262,-0.688254,-0.679553,-0.693609,-0.340935,-0.339523,-0.344480,-0.337773
1,A,1,NF1,WT,7,001,A1,2,6,2,...,-0.533572,-0.532413,-0.641410,-0.651894,-0.656081,-0.621760,-0.504987,-0.498530,-0.507972,-0.503350
2,A,1,NF1,WT,7,001,A1,3,7,3,...,-0.750522,-0.746372,-0.619314,-0.602406,-0.591568,-0.592616,-0.280279,-0.287046,-0.289195,-0.280957
3,A,1,NF1,WT,7,001,A1,4,8,4,...,-0.643813,-0.649677,-0.733028,-0.719157,-0.723398,-0.726127,-0.456826,-0.453214,-0.457110,-0.451529
4,A,1,NF1,WT,7,001,A1,5,9,5,...,-0.547498,-0.541345,-0.701383,-0.697343,-0.694375,-0.699333,-0.360677,-0.370743,-0.374599,-0.371221


## Feature selection

In [9]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

feature_select_norm_sc_pbcellpose = feature_select(
    normalize_sc_pbcellpose,
    operation=feature_select_ops
)

output(feature_select_norm_sc_pbcellpose, sc_norm_fs_output_file_pbcellpose)

print(feature_select_norm_sc_pbcellpose.shape)
feature_select_norm_sc_pbcellpose.head()

(1741, 407)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_InfoMeas2_RFP_3_00_256,Nuclei_Texture_InfoMeas2_RFP_3_01_256,Nuclei_Texture_InfoMeas2_RFP_3_02_256,Nuclei_Texture_InfoMeas2_RFP_3_03_256,Nuclei_Texture_InverseDifferenceMoment_DAPI_3_00_256,Nuclei_Texture_InverseDifferenceMoment_GFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_02_256,Nuclei_Texture_SumEntropy_DAPI_3_02_256,Nuclei_Texture_SumVariance_GFP_3_03_256,Nuclei_Texture_SumVariance_RFP_3_03_256
0,A,1,NF1,WT,7,001,A1,1,5,1,...,1.272383,1.215950,1.159895,1.361630,0.674280,1.375723,2.757122,-0.887113,-0.652033,-0.260070
1,A,1,NF1,WT,7,001,A1,2,6,2,...,0.360350,-0.308577,-0.690463,-0.701700,-0.081629,0.849979,1.393617,-0.393870,-0.574925,-0.456415
2,A,1,NF1,WT,7,001,A1,3,7,3,...,0.788173,-0.351442,-0.632031,0.067306,0.678864,0.446857,-0.771009,-2.253522,-0.573064,-0.277508
3,A,1,NF1,WT,7,001,A1,4,8,4,...,1.102577,0.792408,0.638109,0.777289,0.457330,1.571490,1.836590,-1.116861,-0.681325,-0.392845
4,A,1,NF1,WT,7,001,A1,5,9,5,...,0.676891,-0.157205,-0.358115,-0.061355,0.048907,0.492508,-0.157806,-0.885235,-0.668859,-0.342369


---

### Visualize basic count statistics

In [10]:
sc_df_pbcellpose.Metadata_genotype.value_counts()

Null    1024
WT       717
Name: Metadata_genotype, dtype: int64

In [11]:
pd.crosstab(sc_df_pbcellpose.Metadata_genotype, sc_df_pbcellpose.Metadata_Well)

Metadata_Well,A1,A12,A6,A7,B1,B12,B6,B7,C1,C12,...,F6,F7,G1,G12,G6,G7,H1,H12,H6,H7
Metadata_genotype,,,,,,,,,,,,,,,,,,,,,
Null,0,51,0,58,0,50,0,62,0,66,...,0,78,0,54,0,68,0,52,0,62
WT,53,0,51,0,53,0,51,0,56,0,...,41,0,58,0,38,0,30,0,28,0
